In [2]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import json

nltk.download('punkt')
nltk.download('all')



# Download necessary NLTK packages
#nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /Users/mac/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /Users/mac/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/mac/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /Users/mac/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/mac/nltk_data...
[nltk_data]    |   Package 

True

In [3]:
# Initialize the stemmer and stopwords
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Define the full preprocessing function
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word.isalnum()]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    return tokens

# Directory where the original files are located
original_directory = 'full_docs/'

# Create the inverted index dictionary
inverted_index = {}

# Get a list of only the text files in the original directory
original_filenames = sorted([filename for filename in os.listdir(original_directory) if filename.endswith(".txt")])

# Define batch size (adjust based on your system's capability)
batch_size = 3000

# Process files in batches
for i in range(0, len(original_filenames), batch_size):
    batch_number = i // batch_size
    temp_index_path = f'inverted_index_batch_{batch_number}.json'

    # Check if the batch has already been processed
    if os.path.exists(temp_index_path):
        print(f"Batch {batch_number + 1} already processed. Skipping...")
        continue
    
    # Process each file in the current batch
    batch_filenames = original_filenames[i:i + batch_size]
    for filename in batch_filenames:
        file_path = os.path.join(original_directory, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            tokens = preprocess_text(content)

            for pos, token in enumerate(tokens):
                if token not in inverted_index:
                    inverted_index[token] = {filename: [pos]}
                else:
                    if filename not in inverted_index[token]:
                        inverted_index[token][filename] = [pos]
                    else:
                        inverted_index[token][filename].append(pos)

    # Save the intermediate batch
    with open(temp_index_path, 'w', encoding='utf-8') as index_file:
        json.dump(inverted_index, index_file, indent=4)
    
    print(f"Processed and saved batch {batch_number + 1}")

# Save the final merged inverted index (optional step)
final_index_path = 'inverted_index.json'
with open(final_index_path, 'w', encoding='utf-8') as index_file:
    json.dump(inverted_index, index_file, indent=4)

print(f"Final inverted index with positions saved at: {final_index_path}")


Batch 1 already processed. Skipping...
Batch 2 already processed. Skipping...
Batch 3 already processed. Skipping...
Batch 4 already processed. Skipping...
Batch 5 already processed. Skipping...
Batch 6 already processed. Skipping...
Batch 7 already processed. Skipping...
Batch 8 already processed. Skipping...
Batch 9 already processed. Skipping...
Batch 10 already processed. Skipping...
Batch 11 already processed. Skipping...
Batch 12 already processed. Skipping...
Batch 13 already processed. Skipping...
Batch 14 already processed. Skipping...
Batch 15 already processed. Skipping...
Batch 16 already processed. Skipping...
Batch 17 already processed. Skipping...
Batch 18 already processed. Skipping...
Batch 19 already processed. Skipping...
Batch 20 already processed. Skipping...
Batch 21 already processed. Skipping...
Batch 22 already processed. Skipping...
Batch 23 already processed. Skipping...
Batch 24 already processed. Skipping...
Batch 25 already processed. Skipping...
Batch 26 

In [5]:
def search_inverted_index(query, inverted_index):
    # Preprocess the query (tokenize, lowercase, remove stopwords, etc.)
    query_tokens = preprocess_text(query)

    # Find documents matching each query token
    result_docs = []
    for token in query_tokens:
        if token in inverted_index:
            result_docs.append(set(inverted_index[token]))  # Use sets for intersection

    # If there are results, find the intersection of all token sets (documents that contain all query tokens)
    if result_docs:
        relevant_docs = set.intersection(*result_docs)
        return relevant_docs
    else:
        return set()  # No relevant documents found

# Example usage
query = "does xpress bet charge to deposit money in your account"


inverted_index_path = 'inverted_index.json'

# Load the inverted index from the JSON file
with open(inverted_index_path, 'r', encoding='utf-8') as file:
    inverted_index = json.load(file)

# Search for documents related to the query
results = search_inverted_index(query, inverted_index)

if results:
    print(f"Documents matching the query '{query}': {results}")
else:
    print(f"No documents found for the query '{query}'")

Documents matching the query 'does xpress bet charge to deposit money in your account': {'output_474720.txt'}


In [7]:
import json
import math

# Load the inverted index from its location
#inverted_index_path = '/content/drive/My Drive/inverted_index.json'  # Update with your path
inverted_index_path = 'inverted_index.json'  # Update with your path

with open(inverted_index_path, 'r', encoding='utf-8') as index_file:
    inverted_index = json.load(index_file)

doc_lengths = {}

# Load document lengths and total document count (needed for IDF calculation)
# Calculate document lengths
for term, doc_dict in inverted_index.items():
    for doc, positions in doc_dict.items():
        if doc not in doc_lengths:
            doc_lengths[doc] = len(positions)
        else:
            doc_lengths[doc] += len(positions)
total_docs = len(doc_lengths)  # Total number of unique documents in the collection

# Function to calculate term frequency (TF)
def calculate_tf(term, doc):
    term_count = len(inverted_index[term][doc])
    return (1 + math.log(term_count)) / (1 + math.log(doc_lengths[doc]))  
# Function to calculate inverse document frequency (IDF)
def calculate_idf(term):
    doc_count = len(inverted_index[term])
    return math.log((total_docs + 1) / (doc_count + 1)) + 1 

# Function to handle phrase queries using positional data
def check_phrase_in_document(query_tokens, doc, inverted_index):
    """Check if query tokens appear as a consecutive phrase in the document."""
    positions_list = [inverted_index[token][doc] for token in query_tokens if token in inverted_index and doc in inverted_index[token]]

    if not positions_list or len(positions_list) != len(query_tokens):
        return False 

    # Check for consecutive positions
    for start_pos in positions_list[0]:
        match = True
        for i in range(1, len(positions_list)):
            if (start_pos + i) not in positions_list[i]:
                match = False
                break
        if match:
            return True
    return False

# Function to handle phrase queries
def score_phrase_query(query_tokens, inverted_index):
    relevant_docs = []
    for doc in doc_lengths.keys():
        if check_phrase_in_document(query_tokens, doc, inverted_index):
            relevant_docs.append(doc)
    return relevant_docs

# Function to score documents using TF-IDF with cosine normalization

def calculate_vector_magnitude(vector):
    """Calculate the magnitude of a vector for normalization."""
    return math.sqrt(sum([value ** 2 for value in vector]))

def score_documents(query_tokens, inverted_index):
    scores = {}
    query_term_weights = {}
    query_vector_magnitude = 0

    # Calculate IDF for query terms and build the query vector
    for term in set(query_tokens):
        if term in inverted_index:
            idf = calculate_idf(term)
            query_term_weights[term] = idf
            query_vector_magnitude += idf ** 2

    query_vector_magnitude = math.sqrt(query_vector_magnitude)

    for term in query_tokens:
        if term in inverted_index:
            idf = query_term_weights[term]
            for doc in inverted_index[term]:
                tf = calculate_tf(term, doc)
                tf_idf = tf * idf
                if doc not in scores:
                    scores[doc] = tf_idf ** 2
                else:
                    scores[doc] += tf_idf ** 2

    # Normalize document scores
    for doc in scores:
        doc_vector_magnitude = calculate_vector_magnitude([calculate_tf(term, doc) * query_term_weights[term] for term in query_tokens if term in inverted_index and doc in inverted_index[term]])
        if doc_vector_magnitude != 0 and query_vector_magnitude != 0:
            scores[doc] = scores[doc] / (doc_vector_magnitude * query_vector_magnitude)

    return sorted(scores.items(), key=lambda item: item[1], reverse=True)


# Example usage
#query = "types of road hugger tires"
query = "how much is a cost to run disneyland"


#query = pd.read_csv('/content/drive/My Drive/small_queries.csv')  # Replace with your actual path

query_tokens = preprocess_text(query)  
results = score_documents(query_tokens, inverted_index)

if results:
    print(f"Ranked documents for the query '{query}':")
    for doc, score in results:
        print(f"{doc}: {score:.4f}")
else:
    print(f"No documents found for the query '{query}'")

Ranked documents for the query 'how much is a cost to run disneyland':
output_75217.txt: 0.5417
output_54367.txt: 0.4978
output_501273.txt: 0.4860
output_481720.txt: 0.4811
output_486294.txt: 0.4791
output_85081.txt: 0.4639
output_472434.txt: 0.4483
output_5276.txt: 0.4441
output_470397.txt: 0.4360
output_97344.txt: 0.4314
output_98527.txt: 0.4275
output_500669.txt: 0.4224
output_471391.txt: 0.4192
output_494521.txt: 0.4191
output_490081.txt: 0.4141
output_466908.txt: 0.4028
output_462560.txt: 0.4001
output_469080.txt: 0.3838
output_55174.txt: 0.3772
output_48713.txt: 0.3723
output_95483.txt: 0.3684
output_73853.txt: 0.3571
output_88569.txt: 0.3417
output_67374.txt: 0.3318
output_61817.txt: 0.3234
output_485019.txt: 0.3234
output_466345.txt: 0.3210
output_468713.txt: 0.3180
output_74091.txt: 0.3180
output_491008.txt: 0.3063
output_77713.txt: 0.3055
output_500004.txt: 0.3050
output_491845.txt: 0.3047
output_474130.txt: 0.3034
output_63296.txt: 0.3024
output_62650.txt: 0.2991
output_4857

In [10]:
import pandas as pd
import json
import math

# Load the inverted index
#inverted_index_path = '/content/drive/My Drive/inverted_index.json'
inverted_index_path = 'inverted_index.json'

with open(inverted_index_path, 'r', encoding='utf-8') as index_file:
    inverted_index = json.load(index_file)

# Calculate document lengths (if not already done earlier)
doc_lengths = {}
for term, doc_dict in inverted_index.items():
    for doc, positions in doc_dict.items():
        if doc not in doc_lengths:
            doc_lengths[doc] = len(positions)
        else:
            doc_lengths[doc] += len(positions)
total_docs = len(doc_lengths)

# Functions for TF, IDF, scoring, etc. (use your defined functions)

# Load the CSV file containing the test queries
#queries_df = pd.read_csv('/content/drive/My Drive/small_queries.csv')  
queries_df = pd.read_csv('large_queries.tsv', sep='\t')  


# Store results for each query
all_queries_results = {}

# Process each query and retrieve the top 10 documents
for index, row in queries_df.iterrows():
    query_number = row['Query number']
    query_text = row['Query']

    # Preprocess the query
    query_tokens = preprocess_text(query_text)

    # Score documents
    results = score_documents(query_tokens, inverted_index)

    # Store the top 10 results
    all_queries_results[query_number] = [doc for doc, score in results[:10]]

# Save the results in the required CSV format
output_df = pd.DataFrame([
    {'Query number': query_id, 'Document': doc}
    for query_id, docs in all_queries_results.items()
    for doc in docs
])
output_path = 'results.csv'  
output_df.to_csv(output_path, index=False)

print(f"Test output saved to {output_path}")

Test output saved to results.csv


In [16]:
import json
import glob

inverted_index = {}

# Load each batch file and merge into the final inverted index
batch_files = glob.glob("inverted_index_batch_*.json")
for batch_file in batch_files:
    with open(batch_file, 'r', encoding='utf-8') as index_file:
        batch_data = json.load(index_file)
        for term, postings in batch_data.items():
            if term not in inverted_index:
                inverted_index[term] = postings
            else:
                # Merge postings if term already exists
                for doc, positions in postings.items():
                    if doc not in inverted_index[term]:
                        inverted_index[term][doc] = positions
                    else:
                        inverted_index[term][doc].extend(positions)



total_docs = len(doc_lengths)  # Total number of unique documents

# Function to calculate term frequency (TF)
def calculate_tf(term, doc):
    term_count = len(inverted_index[term][doc])
    return (1 + math.log(term_count)) / (1 + math.log(doc_lengths[doc])) 

# Function to calculate inverse document frequency (IDF)
def calculate_idf(term):
    doc_count = len(inverted_index[term])
    return math.log((total_docs + 1) / (doc_count + 1)) + 1  

# Function to calculate vector magnitude for normalization
def calculate_vector_magnitude(vector):
    return math.sqrt(sum([value ** 2 for value in vector]))

# Function to score documents using TF-IDF with cosine normalization
def score_documents(query_tokens, inverted_index):
    scores = {}
    query_term_weights = {}
    query_vector_magnitude = 0

    # Calculate IDF for query terms and build the query vector
    for term in set(query_tokens):
        if term in inverted_index:
            idf = calculate_idf(term)
            query_term_weights[term] = idf
            query_vector_magnitude += idf ** 2

    query_vector_magnitude = math.sqrt(query_vector_magnitude)

    for term in query_tokens:
        if term in inverted_index:
            idf = query_term_weights[term]
            for doc in inverted_index[term]:
                tf = calculate_tf(term, doc)
                tf_idf = tf * idf
                if doc not in scores:
                    scores[doc] = tf_idf ** 2
                else:
                    scores[doc] += tf_idf ** 2

    # Normalize document scores
    for doc in scores:
        doc_vector_magnitude = calculate_vector_magnitude([
            calculate_tf(term, doc) * query_term_weights[term]
            for term in query_tokens if term in inverted_index and doc in inverted_index[term]
        ])
        if doc_vector_magnitude != 0 and query_vector_magnitude != 0:
            scores[doc] = scores[doc] / (doc_vector_magnitude * query_vector_magnitude)

    return sorted(scores.items(), key=lambda item: item[1], reverse=True)

# Function to calculate Precision at K (P@K)
def precision_at_k(retrieved_docs, relevant_docs, k):
    retrieved_k = retrieved_docs[:k]
    relevant_count = sum([1 for doc in retrieved_k if doc in relevant_docs])
    return relevant_count / k

# Function to calculate Recall at K (R@K)
def recall_at_k(retrieved_docs, relevant_docs, k):
    retrieved_k = retrieved_docs[:k]
    relevant_count = sum([1 for doc in retrieved_k if doc in relevant_docs])
    return relevant_count / len(relevant_docs) if len(relevant_docs) > 0 else 0

# Function to calculate Mean Average Precision at K (MAP@K)
def mean_average_precision_at_k(all_queries_results, all_queries_relevant_docs, k):
    average_precisions = []
    for query_id, retrieved_docs in all_queries_results.items():
        relevant_docs = all_queries_relevant_docs.get(query_id, [])
        precisions = [
            precision_at_k(retrieved_docs, relevant_docs, i + 1)
            for i in range(min(k, len(retrieved_docs)))
            if retrieved_docs[i] in relevant_docs
        ]
        if precisions:
            average_precisions.append(sum(precisions) / len(relevant_docs))
    return sum(average_precisions) / len(average_precisions) if average_precisions else 0

# Function to calculate Mean Average Recall at K (MAR@K)
def mean_average_recall_at_k(all_queries_results, all_queries_relevant_docs, k):
    recalls = []
    for query_id, retrieved_docs in all_queries_results.items():
        relevant_docs = all_queries_relevant_docs.get(query_id, [])
        recall = recall_at_k(retrieved_docs, relevant_docs, k)
        recalls.append(recall)
    return sum(recalls) / len(recalls) if recalls else 0

# Load the CSV file containing the queries (replace with your file path)
#queries_df = pd.read_csv('/content/drive/My Drive/small_queries.csv')  
queries_df = pd.read_csv('large_queries.tsv', sep='\t')  


# Load your result.csv containing the ground truth data
#ground_truth_path = ('/content/drive/My Drive/results.csv') 
ground_truth_path = ('results.csv')  

ground_truth_df = pd.read_csv(ground_truth_path)

# Create the ground truth dictionary
all_queries_relevant_docs = {}
for _, row in ground_truth_df.iterrows():
    query_number = row['Query number']
    document = row['Document']

    if query_number not in all_queries_relevant_docs:
        all_queries_relevant_docs[query_number] = [document]
    else:
        all_queries_relevant_docs[query_number].append(document)


# Iterate over each query and run the retrieval system
for index, row in queries_df.iterrows():
    query_number = row['Query number']
    query_text = row['Query']

    # Preprocess the query text
    query_tokens = preprocess_text(query_text)

    # Get the ranked results for the query
    results = score_documents(query_tokens, inverted_index)

    # Store the top 10 results for each query
    all_queries_results[query_number] = [doc for doc, score in results[:10]]

# Compute MAP@3, MAP@10, MAR@3, MAR@10
map_at_3 = mean_average_precision_at_k(all_queries_results, all_queries_relevant_docs, 3)
map_at_10 = mean_average_precision_at_k(all_queries_results, all_queries_relevant_docs, 10)
mar_at_3 = mean_average_recall_at_k(all_queries_results, all_queries_relevant_docs, 3)
mar_at_10 = mean_average_recall_at_k(all_queries_results, all_queries_relevant_docs, 10)

print(f"MAP@3: {map_at_3:.4f}")
print(f"MAP@10: {map_at_10:.4f}")
print(f"MAR@3: {mar_at_3:.4f}")
print(f"MAR@10: {mar_at_10:.4f}")

KeyboardInterrupt: 